# Вебинар 5. Домашнее задание

Само домашнее задание нахожится в конце ноутбука

---

In [98]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import ItemItemRecommender  # нужен для одного трюка
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

# Функции из 1-ого вебинара
import os, sys

module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from src.metrics import precision_at_k, recall_at_k
from src.utils import prefilter_items

In [99]:
data = pd.read_csv('./raw_data/transaction_data.csv')

data.columns = [col.lower() for col in data.columns]
data.rename(columns={'household_key': 'user_id',
                    'product_id': 'item_id'},
           inplace=True)


test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(2)

user_id    basket_id  day    item_id  quantity  sales_value  store_id  \
0     2375  26984851472  1.0  1004906.0       1.0         1.39     364.0   
1     2375  26984851472  1.0  1033142.0       1.0         0.82     364.0   

   retail_disc  trans_time  week_no  coupon_disc  coupon_match_disc  
0         -0.6      1631.0      1.0          0.0                0.0  
1          0.0      1631.0      1.0          0.0                0.0

In [100]:
item_features = pd.read_csv('./raw_data/product.csv')
item_features.columns = [col.lower() for col in item_features.columns]
item_features.rename(columns={'product_id': 'item_id'}, inplace=True)

item_features.head(2)

item_id  manufacturer    department     brand            commodity_desc  \
0    25671             2       GROCERY  National                  FRZN ICE   
1    26081             2  MISC. TRANS.  National  NO COMMODITY DESCRIPTION   

            sub_commodity_desc curr_size_of_product  
0          ICE - CRUSHED/CUBED                22 LB  
1  NO SUBCOMMODITY DESCRIPTION

In [101]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

user_id                                             actual
0        1  [836548.0, 856942.0, 877391.0, 933913.0, 94842...
1        2  [826784.0, 827176.0, 830127.0, 834826.0, 86814...

In [104]:
n_items_before = data_train['item_id'].nunique()

data_train = prefilter_items(data_train, item_features, take_n_popular=5000)

n_items_after = data_train['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

Decreased # items from 82583 to 5000


In [105]:
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробоват ьдругие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

user_item_matrix.head(3)

item_id  818980.0    818981.0    819063.0    819255.0    819304.0    \
user_id                                                               
1               0.0         0.0         0.0         0.0         0.0   
2               0.0         0.0         0.0         0.0         0.0   
3               0.0         0.0         0.0         1.0         0.0   

item_id  819308.0    819330.0    819518.0    819594.0    819765.0    ...  \
user_id                                                              ...   
1               0.0         0.0         0.0         0.0         0.0  ...   
2               0.0         0.0         0.0         0.0         0.0  ...   
3               0.0         0.0         0.0         0.0         0.0  ...   

item_id  15511891.0  15596279.0  15596488.0  15596515.0  15629778.0  \
user_id                                                               
1               0.0         0.0         0.0         0.0         0.0   
2               0.0         0.0         0.0         0.0         0.0   
3               0.0         2.0         0.0         0.0         0.0   

item_id  15630096.0  15926844.0  15926886.0  15927403.0  15927850.0  
user_id                                                              
1               0.0         1.0         0.0         0.0         0.0  
2               0.0         0.0         0.0         0.0         0.0  
3               0.0         0.0         0.0         0.0         0.0  

[3 rows x 5000 columns]

In [106]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [107]:
user_item_matrix = bm25_weight(user_item_matrix.T).T  # Применяется к item-user матрице ! 

In [148]:
%%time

model = AlternatingLeastSquares(factors=20, 
                                regularization=0.001,
                                iterations=15, 
                                calculate_training_loss=True, 
                                num_threads=4)

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)


CPU times: user 13.2 s, sys: 16.4 s, total: 29.6 s
Wall time: 9.99 s


-----

# Домашнее задание

1. Изучите структуру модуля src
2. Перенесите функции prefilter_items и postfilter_items из вебинара в модуль src.utils.py
3. Реализуйте функции get_similar_items_recommendation, get_similar_users_recommendation (они разбирались на вебинаре) и переместите в src.utils.py
4. Создайте модуль src.recommenders.py. Напищите код для класса ниже и положите его в src.recommenders.py
5. Проверьте, что все модули корректно импортируются
6. Если вы еще не прочитали [статью](https://habr.com/ru/company/hh/blog/347276/) о рекомендательных системах и поиске в hh.ru, то обязательно прочитайте

In [103]:
#изменил работу функции prefilter_items таким образом, чтобы этот файл мог запускаться и перенес в src.utils.py
def prefilter_items(data, item_features, take_n_popular=0):
    popularity = data_train.groupby('item_id')['user_id'].nunique().reset_index()
    popularity['user_id'] = popularity['user_id']/ data_train['user_id'].nunique()
    popularity.rename(columns={'user_id': 'share_unique_users'}, inplace=True)
    #deleted very popular and unpopular items
    take_n = popularity[(popularity['share_unique_users'] < 0.5 ) & (popularity['share_unique_users'] > 0.01 )] 
     
    if take_n_popular != 0:
        #in case if we would like to reduce our sample we sort items in decending order to keep the most popular one
        take_n = take_n.sort_values('share_unique_users', ascending = False)
        take_n = take_n[:take_n_popular].item_id.tolist()
    else:
        take_n = take_n.item_id.tolist()
        
    data = data[data['item_id'].isin(take_n)]
    #added additional information about items
    full_data = pd.merge(data, item_features, on='item_id')
    return full_data 

In [109]:
def postfilter_items(user_id, recommednations):
    pass

In [144]:
popularity = data_train.groupby(['user_id', 'item_id'])['quantity'].count().reset_index()
popularity.sort_values('quantity', ascending=False, inplace=True)

#popularity = popularity[popularity['item_id'] != 999999]

popularity = popularity.groupby('user_id').head(5)

popularity.sort_values('user_id', ascending=False, inplace=True)
popularity.head(2)

user_id    item_id  quantity
634081     2500  1058997.0         7
634184     2500  5978656.0         6

In [157]:
def get_similar_items_recommendation(user, model, N=5):
    """Рекомендуем товары, похожие на топ-N купленных юзером товаров"""
    recs = model.similar_items(itemid_to_id[user], N=N)
    all_recs= [id_to_itemid[rec[0]] for rec in recs]
    return  all_recs 

def get_similar_users_recommendation(user, model, N=5):
    """Рекомендуем топ-N товаров, среди купленных похожими юзерами"""
    try:
        res = [id_to_itemid[rec[0]] for rec in model.recommend(userid=userid_to_id[user], 
                                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                                        N=N, 
                                        filter_already_liked_items=False, 
                                        #filter_items=[itemid_to_id[999999]],  # !!! have already deleted in def prefilter_items
                                        recalculate_user=True)]
    except KeyError: 
        res = None

    return res

In [117]:
result['bm25'] = result['user_id'].apply(lambda x: get_similar_items_recommendation(x, model=model, N=5))
result = result[result['bm25'].notna()]

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [160]:
popularity['similar_recommendation'] = popularity['item_id'].apply(lambda x: get_similar_items_recommendation(x, model, N=5))
popularity.head(3)

user_id    item_id  quantity  \
634081     2500  1058997.0         7   
634184     2500  5978656.0         6   
634190     2500  6534178.0        10   

                                   similar_recommendation  
634081  [1058997.0, 1126899.0, 1055853.0, 970119.0, 86...  
634184  [5978656.0, 5978648.0, 6534178.0, 1027569.0, 9...  
634190  [6534178.0, 5978656.0, 5978648.0, 882305.0, 10...

In [130]:
#ERROR:root: UnicodeDecodeError while processing traceback. если пользоваться from src.metrics import precision_at_k 
def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  # “ут нет [:k] !!
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    #print(flags)
    
    precision = flags.sum() / len(recommended_list)
    return precision

In [132]:
result.apply(lambda row: precision_at_k(row['bm25'], row['actual']), axis=1).mean()

0.11477907567292986

In [168]:
import pandas as pd
import numpy as np

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import ItemItemRecommender  # нужен для одного трюка
from implicit.nearest_neighbours import bm25_weight, tfidf_weight


class MainRecommender:
    """Рекоммендации, которые можно получить из ALS
    
    Input
    -----
    user_item_matrix: pd.DataFrame
        Матрица взаимодействий user-item
    """
    
    def __init__(self, data, weighting=True):
        
        # your_code. Это не обязательная часть. Но если вам удобно что-либо посчитать тут - можно это сделать
        
        self.user_item_matrix = self.prepare_matrix(data)  # pd.DataFrame
        self.id_to_itemid, self.id_to_userid, self.itemid_to_id, self.userid_to_id = prepare_dicts(self.user_item_matrix)
        
        if weighting:
            self.user_item_matrix = bm25_weight(self.user_item_matrix.T).T 
        
        self.model = self.fit(self.user_item_matrix)
        self.own_recommender = self.fit_own_recommender(self.user_item_matrix)
     
    @staticmethod
    def prepare_matrix(data):
        # your_code
        user_item_matrix = pd.pivot_table(data, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробоват ьдругие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

        user_item_matrix = user_item_matrix.astype(float)
        return user_item_matrix
    
    @staticmethod
    def prepare_dicts(user_item_matrix):
        """Подготавливает вспомогательные словари"""
        
        userids = user_item_matrix.index.values
        itemids = user_item_matrix.columns.values

        matrix_userids = np.arange(len(userids))
        matrix_itemids = np.arange(len(itemids))

        id_to_itemid = dict(zip(matrix_itemids, itemids))
        id_to_userid = dict(zip(matrix_userids, userids))

        itemid_to_id = dict(zip(itemids, matrix_itemids))
        userid_to_id = dict(zip(userids, matrix_userids))
        
        return id_to_itemid, id_to_userid, itemid_to_id, userid_to_id
     
    @staticmethod
    def fit_own_recommender(user_item_matrix):
        """Обучает модель, которая рекомендует товары, среди товаров, купленных юзером"""
    
        own_recommender = ItemItemRecommender(K=1, num_threads=4)
        own_recommender.fit(csr_matrix(user_item_matrix).T.tocsr())
        
        return own_recommender
    
    @staticmethod
    def fit(user_item_matrix, n_factors=20, regularization=0.001, iterations=15, num_threads=4):
        """Обучает ALS"""
        
        model = AlternatingLeastSquares(factors=factors, 
                                             regularization=regularization,
                                             iterations=iterations,  
                                             num_threads=num_threads)
        model.fit(csr_matrix(self.user_item_matrix).T.tocsr())
        
        return model

    def get_similar_items_recommendation(self, user, N=5):
        """Рекомендуем товары, похожие на топ-N купленных юзером товаров"""

        # your_code
        
        recs = model.similar_items(itemid_to_id[user], N=N)
        recs= [id_to_itemid[rec[0]] for rec in recs]
        

        assert len(recs) == N, 'Количество рекомендаций != {}'.format(N)
        return  recs
    
    def get_similar_users_recommendation(self, user, N=5):
        """Рекомендуем топ-N товаров, среди купленных похожими юзерами"""
    
        # your_code
        try:
            res = [id_to_itemid[rec[0]] for rec in model.recommend(userid=userid_to_id[user], 
                                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                                        N=N, 
                                        filter_already_liked_items=False, 
                                        #filter_items=[itemid_to_id[999999]],  # !!! have already deleted in def prefilter_items
                                        recalculate_user=True)]
        except KeyError: 
            res = None

        assert len(res) == N, 'Количество рекомендаций != {}'.format(N)
        return res

----

Проверка, что все работает

In [ ]:
from src.metrics import precision_at_k, recall_at_k
from src.utils import prefilter_items
from src.recommenders import MainRecommender

